In [ ]:
!pip install google-generativeai pandas

In [ ]:
import requests
import pandas as pd
import time
import os
from getpass import getpass

In [ ]:
API_KEY = getpass("Insira sua chave da API do Gemini:")


Insira sua chave da API do Gemini:··········


In [ ]:
URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={API_KEY}"

In [ ]:
pasta_entrada = "./DATA-SPLIT/"
pasta_saida = "./Gemini/"

os.makedirs(pasta_entrada, exist_ok=True)
os.makedirs(pasta_saida, exist_ok=True)

In [ ]:
import requests
import time

# Função para classificar uma frase
def classificar_frase(frase):
    prompt = f"""
    Você é um modelo de IA treinado para classificar frases como:

    - "0" se a frase contém discurso de ódio (ataques direcionados a grupos com base em raça, religião, orientação sexual, etc.).
    - "1" se a frase contém linguagem ofensiva, mas não for discurso de ódio.
    - "2" se a frase não contém discurso de ódio nem linguagem ofensiva

    Regras:
    - Responda *apenas* com "0", "1" ou "2".
    - Não adicione explicações ou qualquer outro texto.

    Frase: "{frase}"
    """
    # print(f"Classificando frase: {frase[:50]}...") # Imprime as primeiras 50 letras da frase

    try:
        resp = requests.post(URL, json={"contents": [{"parts": [{"text": prompt}]}]}, headers={"Content-Type": "application/json"})
        time.sleep(5)

        if resp.status_code == 200:
            resposta = resp.json()
            # print(f"Resposta da API: {resposta}") # Imprime a resposta completa da API
            texto = resposta.get("candidates", [{}])[0].get("content", {}).get("parts", [{}])[0].get("text", "").strip()
            # print(f"Texto extraído da resposta: {texto}") # Imprime o texto extraído da resposta

            # Garante que a resposta seja apenas "0", "1" ou "2"
            if texto.startswith("0"):
                return "0"
            elif texto.startswith("1"):
                return "1"
            elif texto.startswith("2"):
                return "2"
            else:
                print(f"Resposta inesperada da API: {texto}")
                return "Erro"

        else:
            print(f"Erro API {resp.status_code}: {resp.text}")
            return f"Erro API {resp.status_code}"

    except Exception as e:
        print(f"Exceção na função classificar_frase: {e}")
        return f"Erro: {e}"

In [ ]:
def processar_excel(pasta_entrada, pasta_saida):
    for i in range(1, 2):  # de 1 até 11
        entrada = f"{pasta_entrada}/amostras.xlsx"
        saida = f"{pasta_saida}/gemini_amostras.xlsx"

        if os.path.exists(entrada):
            df = pd.read_excel(entrada)
            if "tweet" in df.columns:
                df["Gemini"] = df["tweet"].apply(classificar_frase)
                df.to_excel(saida, index=False)
                print(f"Salvo: {saida}")
            else:
                print(f"Erro: coluna 'tweet' não encontrada em {entrada}")


In [ ]:
processar_excel(pasta_entrada, pasta_saida)

Salvo: ./Gemini//gemini_amostras.xlsx
